In [23]:
import os
import time

from functions.input2dataframe import *

from functions.analyse_df_pcap import *
from functions.tcpdump_filter_anon import *

from functions.analyse_df_nfdump import *
from functions.nfdump_filter import *

In [36]:
%run 'functions/analyse_df_pcap'

In [46]:
def ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True):
    
    start = time.clock()
    if debug: print 'Converting input file to dataframe (time:',  
    df = input2dataframe(input_file) #input2dataframe checks the input_file type (pcap, pcapng, nfdump) and calls the right converter (e.g. pcap2dataframe, nfdump2dataframe)
#     display(df[df['dst_ip']=='185.188.206.99']['dst_port'].value_counts())
    if debug: print str(time.clock() - start)+')'
    
    #########################################################        
    #########################################################
    start = time.clock()
    if debug: print 'Analysing the dataframe for finding attack patterns...'    
        
    if 'tuple' in str(type(df)):
        all_patterns = analyse_df_nfdump(df, debug)
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.clock() - start)+')'
        #########################################################        
        #########################################################
        start = time.clock()
        if debug: print 'Generating attack vectors (with the same format as the input file) and anonymizing the victim information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            nfdump_filter(input_file, all_patterns['dst_ip'], pattern)
        #########################################################        
        #########################################################   
    else:
        all_patterns = analyse_df_pcap(df, debug=True)
        display(all_patterns)
        
        if debug: print 'Dataframe completely analysed and all attack patterns identified! (time:'+str(time.clock() - start)+')'
        #########################################################        
        #########################################################
        start = time.clock()
        if debug: print 'Generating attack vectors (same format as input file) and anonymizing victim\'s information (dst_ip) (time:' 
        for pattern in all_patterns["patterns"]:
            tcpdump_filter_anon(input_file, pattern, all_patterns["dst_ip"])
            
    if debug: print str(time.clock() - start)+')'
#     #########################################################        
#     #########################################################
#     #TODO: Generate rule/signatures for specific detection and mitigation technologies (SIDS: BRO, SNORT, SURICATA; Firewall)
    

## Execution from Jupyter Notebook

In [47]:
input_file="input_file_for_test/mirai.pcap"
ddos_analysis_fingerprint_filter_sanitization(input_file)

Converting input file to dataframe (time: 3.415241)
Analysing the dataframe for finding attack patterns...
Total number packets: 92148

###################################
IDENTIFYING MAIN CHARACTERISTICS:
###################################
Target (destination) IP: 185.188.206.99
Number of packets: 92075

###################################################################################################################
IP protocol used in packets going to target IP: 6
Number of packets: 92075

#############################
PORT FREQUENCY OF REMAINING PACKETS
##############################
SOURCE ports frequency
54046    0.008689
26726    0.008689
5426     0.008689
63714    0.008689
35443    0.008689
Name: src_port, dtype: float64

DESTINATION ports frequency
21325    0.009775
25124    0.009775
15269    0.009775
1779     0.008689
62820    0.008689
Name: dst_port, dtype: float64

 Using top dest port:  21325

##############################################################################

{'dst_ip': '185.188.206.99', 'patterns': []}

Dataframe completely analysed and all attack patterns identified! (time:0.111787)
Generating attack vectors (same format as input file) and anonymizing victim's information (dst_ip) (time:
0.000111000000004)


## Main for python script 

In [4]:
if __name__ == '__main__':
    import argparse
    import os.path
    
    parser = argparse.ArgumentParser(description='')
    
    parser.add_argument('--input', metavar='input_file', required=True,
                        help='Path of a input file')
    args = parser.parse_args()
    
    input_file=args.input
    
    if os.path.isfile(input_file) == False:
        print 'We were unable to find the file. Please check the file path!!'
    
        ddos_analysis_fingerprint_filter_sanitization(input_file, debug=True)

usage: ipykernel_launcher.py [-h] --input input_file
ipykernel_launcher.py: error: argument --input is required


SystemExit: 2

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
